In [35]:
# ChromeDriver 자동 설치 모듈
import chromedriver_autoinstaller
# Chrome을 제어하기 위한 객체
from selenium import webdriver
# Chrome이 웹 페이지 로딩을 완료 할 때까지 최대 n초간 대기하는 기능.
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from pandas import DataFrame
# 파이썬 프로그램에 지정된 시간동안 랙을 거는 기능을 위해 사용
import time

In [36]:
#크롬드라이버 자동 설치
chromedriver_autoinstaller.install()
# 크롬드라이버를 통해 크롬을 실행시킴
# -> driver 객체는 Chrome 자체
driver = webdriver.Chrome()
# 크롬브라우저가 준비될 때 까지 최대 5초씩 대기
driver.implicitly_wait(5)

In [37]:
# -> URL을 분석하여 얻어낸 값
params = ['mon','tue','wed','thu','fri','sat','sun','dailyPlus']
# 네이버 웹툰의 주소 형식
naverWebtoonUrl = "https://comic.naver.com/webtoon?tab={0}"

In [38]:
# 수집된 결과가 누적될 빈 리스트
naverWebtoonData = []

# 요일별 반복
for p in params:
    # 특정 요일의 네이버 웹툰 페이지
    url = naverWebtoonUrl.format(p)
    #print(url)
    # 크롬 브라우저로 방문시킴
    driver.get(url)
    time.sleep(1)

    # 브라우저에 표시되는 전체 코드를 추출 ---> bs4 객체로 변환
    soup = BeautifulSoup(driver.page_source)
    webtoonList = soup.select(".ContentList__content_list--q5KXY > .item")

    # 추출된 웹툰 목록 수만큼 반복
    for w in webtoonList:
        # 포스터 URL 가져오기
        poster = w.select('Poster__image--d9XTI')

        # if 가져온 이미지가 존재하고, src 속성이 있다면?
        if poster and 'src' in poster[0].attrs:
            posterValue = poster[0].attrs['src']
        else:
            posterValue = None  
        
        url = w.select('.Poster__link--sopnC')
        if url and 'href' in url[0].attrs:
            urlValue = url[0].attrs['href']
        
        if urlValue.find("https://comic.naver.com") == -1:
            urlValue = "https://comic.naver.com" + urlValue
        
        # 웹툰 제목 가져오기
        webtoonTitle = w.select(".ContentTitle__title--e3qXt > .text")
        if webtoonTitle:
            titleValue = webtoonTitle[0].text.strip()
        else:
            titleValue = None
        
        author = w.select(".ContentAuthor__author--CTAAP")

        if author:
            authorValue = author[0].text.strip()
        else:
            authorValue = None
        rating = w.select(".Rating__star_area--dFzsb > .text")

        if rating:
            ratingValue = rating[0].text.strip()
        else:
            ratingValue = None

        resultDict = {"요일":p,"제목":titleValue,"작가":authorValue, "별점":ratingValue}
        naverWebtoonData.append(resultDict)

In [40]:
df = DataFrame(naverWebtoonData)
df.to_excel('요일별_네이버_웹툰.xlsx')
df

,요일,제목,작가,별점
0,mon,뷰티풀 군바리,설이 / 윤성원,9.81
1,mon,신의 탑,SIU,9.88
2,mon,퀘스트지상주의,박태준 만화회사,9.76
3,mon,똑 닮은 딸,이담,9.98
4,mon,백수세끼,치즈,9.84
...,...,...,...,...
721,dailyPlus,후드,후렛샤 / 심규호,8.34
722,dailyPlus,실버벨,노떼,9.53
723,dailyPlus,씨앗의 정원,백희,9.69
724,dailyPlus,도와줘우주,설레임 / 한겨울,9.33
